<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202402/blob/main/personalidad/01-script-ETL-personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Tema 01: Carga de datos**

### **Importando base de datos**

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API Client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = "https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing"
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet("Respuestas de formulario 1").get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

### **Inspección data frame**

In [ ]:
type(df)

In [ ]:
# Ver cantidad de filas y columnas
df.shape

In [ ]:
# Ver primeras filas
df.head()

In [ ]:
# Ver info general y nombre de columas
df.info()

In [ ]:
# Tipo de datos
df['Sexo'].dtype

In [ ]:
# Número de filas
len(df)

In [ ]:
# Número de columnas
len(df.columns)

### **Tema 02: Transformación de datos**

### **Valores perdidos**

#### **Identificación de los NAs**

In [ ]:
# Los NAs están strings vacíos
import numpy as np
df.replace("", np.nan, inplace=True)

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

#### **Tratando de los NAs**

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"] = pd.to_numeric(df["Escribe tu edad exacta"], errors="coerce")

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"].mean()

In [ ]:
edad_promedio = df["Escribe tu edad exacta"].mean()

##### **Reemplazo por la media**

In [ ]:
# Creando df2
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2["edad2"] = df2["Escribe tu edad exacta"].fillna(edad_promedio)

In [ ]:
df2[["Escribe tu edad exacta", "edad2"]]

**Reubicando edad2**

In [ ]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [ ]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove("edad2")

In [ ]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index("Escribe tu edad exacta")

In [ ]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index("Escribe tu edad exacta") + 1, "edad2")

In [ ]:
# Reordenando y guardando en df2
df2 = df2[lista_columnas]

In [ ]:
df2.head()

**Creando relocate() en python**

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = "edad2",
    after = "Escribe tu edad exacta"
)

##### **Eliminar la fila completa**

In [ ]:
df2.shape

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

### **Estandarización de variables**

#### **Normalización**

In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
# instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[["edad2"]])

In [71]:
# creando df3
df3 = df2.copy()

In [72]:
df3["edadZ"] = normalizador.fit_transform(df3[["edad2"]])

In [73]:
# reubicando edadZ despues de edad2
df3 = relocate(
    df = df3,
    columna = "edadZ",
    after = "edad2"
)

In [ ]:
df3[["edad2", "edadZ"]]

#### **Rango**

In [77]:
from sklearn.preprocessing import MinMaxScaler

In [78]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [79]:
# convirtiendo a rango
df3["edadRango"] = rango.fit_transform(df3[["edad2"]])

In [80]:
df3 = relocate(
    df = df3,
    columna = "edadRango",
    after = "edadZ"
)

In [83]:
df3[["edad2", "edadRango"]]

,edad2,edadRango
0,16.0,0.058824
1,16.0,0.058824
2,17.0,0.088235
3,22.0,0.235294
4,16.0,0.058824
...,...,...
233,20.0,0.176471
234,23.0,0.264706
235,22.0,0.235294
236,27.0,0.382353
